In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/Preprocessed_Data.csv")

In [3]:
df

,order_id,R_Lat,R_Lon,C_Lat,C_Lon,order_pickedup_time,order_delivered_time,Distance (km),Delivery_Hour,State,Delivery_Time (sec)
0,PRD_ZMT_2960011,28.468983,77.063728,28.449929,77.071230,2016-04-23 09:30:20,2016-04-23 09:57:04,2.242045,9,Haryana,1604.0
1,PRD_ZMT_2964555,28.468983,77.063728,28.454522,77.066637,2016-04-23 11:05:32,2016-04-23 11:27:10,1.632941,11,Haryana,1298.0
2,PRD_ZMT_2968945,28.468983,77.063728,28.461570,77.067590,2016-04-23 14:49:07,2016-04-23 15:07:18,0.906626,14,Haryana,1091.0
3,PRD_ZMT_2976663,28.468983,77.063728,28.455738,77.058401,2016-04-23 16:43:14,2016-04-23 17:16:12,1.562127,16,Haryana,1978.0
4,PRD_ZMT_2985948,28.468983,77.063728,28.440068,77.083967,2016-04-24 07:54:14,2016-04-24 08:16:29,3.775236,7,Haryana,1335.0
...,...,...,...,...,...,...,...,...,...,...,...
237368,PRD_ZMT_8457746,28.532886,77.207941,28.503037,77.201892,2016-09-25 16:49:02,2016-09-25 17:18:55,3.371266,16,Delhi,1793.0
237369,PRD_ZMT_8458181,22.544237,88.352148,22.560074,88.369037,2016-09-25 17:07:48,2016-09-25 17:22:04,2.471663,17,West Bengal,856.0
237370,PRD_ZMT_8458897,19.122455,72.916411,19.104956,72.918557,2016-09-25 17:22:39,2016-09-25 17:43:55,1.958820,17,Maharashtra,1276.0
237371,PRD_ZMT_8458955,17.450286,78.379158,17.444102,78.391155,2016-09-25 17:07:15,2016-09-25 17:26:16,1.446523,17,Telangana,1141.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237373 entries, 0 to 237372
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              237373 non-null  object 
 1   R_Lat                 237373 non-null  float64
 2   R_Lon                 237373 non-null  float64
 3   C_Lat                 237373 non-null  float64
 4   C_Lon                 237373 non-null  float64
 5   order_pickedup_time   237373 non-null  object 
 6   order_delivered_time  237373 non-null  object 
 7   Distance (km)         237373 non-null  float64
 8   Delivery_Hour         237373 non-null  int64  
 9   State                 237373 non-null  object 
 10  Delivery_Time (sec)   237373 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 19.9+ MB


In [5]:
# For incomplete data
# df = df[:194000]
# df

# Label Encoding of `State` column

In [6]:
df["State"].unique()

array(['Haryana', 'Delhi', 'Maharashtra', 'Tamil Nadu', 'Telangana',
       'West Bengal', 'Karnataka', 'Borno'], dtype=object)

In [7]:
from sklearn import preprocessing

In [8]:
label_encoder = preprocessing.LabelEncoder()

df['State'] = label_encoder.fit_transform(df['State'])

df['State'].unique()

array([2, 1, 4, 5, 6, 7, 3, 0])

# Let's split the dataset in Train & Test dataset

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x = np.array(df[["Distance (km)", "State", "Delivery_Hour"]])
y = np.array(df["Delivery_Time (sec)"])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Now, we are going to train the model. For that purpose, we are going to use LSTM

In [12]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

2024-04-10 09:33:50.294204: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 09:33:51.652688: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 09:33:54.515605: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 09:34:01.277789: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

2024-04-10 09:34:08.299247: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 09:34:08.362846: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/soumya/PyCharmProjects/NewsBytes_Assignment/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the fir

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 3, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=100)

Epoch 1/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - loss: 4208574.5000
Epoch 2/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 3442019.7500
Epoch 3/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 3867055.0000
Epoch 4/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - loss: 2836346.7500
Epoch 5/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 3322314.2500
Epoch 6/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 5428532.0000
Epoch 7/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 4597876.0000
Epoch 8/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 3429303.2500
Epoch 9/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - loss: 2322016.5000
Epoch 10/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - loss: 4865373.0000
Epoch 11/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - loss: 4828482.0000
Epoch 12/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - loss: 3050151.7500
Epoch 13/50
1899/1899 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - loss: 5443981.0000
Epoch

In [15]:
model.evaluate(X_test, y_test)

1484/1484 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 2877135.7500


2161377.5

In [16]:
model.predict(np.array([[2.242045, 1, 9]]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step


array([[1287.1779]], dtype=float32)